## # Introduction
<p><img src="https://i.imgur.com/kjWF1So.jpg" alt="Different characters on a computer screen"></p>
<p>According to a 2019 <a href="https://storage.googleapis.com/gweb-uniblog-publish-prod/documents/PasswordCheckup-HarrisPoll-InfographicFINAL.pdf">Google / Harris Poll</a>, 24% of Americans have used common passwords, like <code>abc123</code>, <code>Password</code>, and <code>Admin</code>. Even more concerning, 59% of Americans have incorporated personal information, such as their name or birthday, into their password. This makes it unsurprising that 4 in 10 Americans have had their personal information compromised online. Passwords with commonly used phrases and personal information makes cracking a password drastically easier.</p>
<p>You may have noticed over the years that password requirements have increased in complexity, including recommendations to change your passwords every couple of months. Compiled from industry recommendations, below is a list of passwords requirements you will be asked to test: </p>
<p><strong>Password Requirments:</strong></p>
<ol>
<li>Must be at least 10 characters in length</li>
<li>Must contain at least:<ul>
<li>one lower case letter </li>
<li>one upper case letter </li>
<li>one numeric character </li>
<li>one non-alphanumeric character</li></ul></li>
<li>Must not contain the phrase <code>password</code> (case insensitive)</li>
<li>Must not contain the user's first or last name, e.g., if the user's name is <code>John Smith</code>, then <code>SmItH876!</code> is not a valid password.</li>
</ol>
<p>Here is the dataset that you will investigate this project:</p>
<div style="background-color: #ebf4f7; color: #595959; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/logins.csv</b></div>
Each row represents a login credential. There are no missing values and you can consider the dataset "clean".
<ul>
    <li><b>id:</b> the user's unique ID.</li>
    <li><b>username:</b> the username with the format {firstname}.{lastname}.</li>
    <li><b>password:</b> the password that may or may not meet the requirements. <i>Note, passwords should never be saved in plaintext, always encrypt them when working with real live passwords!</i></li>
</ul>
</div>
<p>Warning: This dataset contains some <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have been filtered, but may still include words that are explicit and offensive.</p>
<p>From here on out, it will be your task to explore and manipulate the existing data until you can answer the two questions described in the instructions panel. Feel free to import as many packages as you need to complete your task, and add cells as necessary. Finally, remember that you are only tested on your answer, not on the methods you use to arrive at the answer!</p>
<p><strong>Note:</strong> To complete this project, you need to know how to manipulate strings in pandas DataFrames and be familiar with regular expressions. Before starting this project we recommend that you have completed the following courses: <a href="https://learn.datacamp.com/courses/data-cleaning-in-python">Data Cleaning in Python</a> and <a href="https://learn.datacamp.com/courses/regular-expressions-in-python">Regular Expressions in Python</a>.</p>

In [20]:
# Use this cell to begin your analysis, and add as many as you would like!
# Import pandas
import pandas as pd

# load dataset as logins
logins = pd.read_csv('datasets/logins.csv')

# Examine our dataset, info + head
display(logins.info())
logins.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 3 columns):
id          982 non-null int64
username    982 non-null object
password    982 non-null object
dtypes: int64(1), object(2)
memory usage: 23.1+ KB


None

,id,username,password
0,1,vance.jennings,vanceRules888!
1,2,consuelo.eaton,Mail_Pen%Scarlets.414
2,3,mitchel.perkins,Z00+1960
3,4,odessa.vaughan,D-rockyou
4,5,araceli.wilder,Araceli}r3


In [21]:
# Great, we dont have any NULL cell

# Now we will start with the first requirement
# (Must be at least 10 characters in length)
# We will assign a variable (length),
# it will be True if the password length < 10
length = logins.password.str.len() < 10

# Second requirement: must contains at least (one lower case letter, 
# one upper case letter, one numeric character, one non-alphanumeric character)
# We will assaign variable (letters), it will be True if any condition failed
letters = ~(logins.password.str.contains('[a-z]') & # at least one lower case
        logins.password.str.contains('[A-Z]') &     # at least one upper case
        logins.password.str.contains('\d') &        # at least one numaric
        logins.password.str.contains('\W'))         # at least non-alphanumeric

# Third requirement: must not contain the phrase password (case insensitive) 
# We will assaign variable (phrase), it will be True if have (password) phrase
phrase = logins.password.str.contains('password', case=False)

# Forth requirement:Must not contain user's first or last name(case insensitive)
# Extracting first and last names in seperate variables
first_name = logins.username.str.extract('(^[a-z]+)', expand = False)
last_name = logins.username.str.extract('([a-z]+$)', expand = False)
# We will iterate each row to check if first or last name appears in password
names = []
for index,row in logins.iterrows():
    if (first_name[index] in row.password.lower()
       ) | (last_name[index] in row.password.lower()):
        name = True
    else: 
        name = False
    
    names.append(name)

names = pd.Series(names)        

# Now we will creat a column (invalid_password), it will be True if any 
# requirements failed 
logins['invalid_password'] = (length | letters | phrase | names)

# Let us see the first 10 names in our dataframe
logins.head(10)

,id,username,password,invalid_password
0,1,vance.jennings,vanceRules888!,True
1,2,consuelo.eaton,Mail_Pen%Scarlets.414,False
2,3,mitchel.perkins,Z00+1960,True
3,4,odessa.vaughan,D-rockyou,True
4,5,araceli.wilder,Araceli}r3,True
5,6,shawn.harrington,126_239_123,True
6,7,evelyn.gay,`4:&iAt$'o~(,False
7,8,noreen.hale,25941829163,True
8,9,gladys.ward,=Wj1`i)xYYZ,False
9,10,brant.zimmerman,L?4)OSB$r,True


In [22]:
# Put the bad password data alone
bad_pwd_data = logins[logins.invalid_password == True].sort_values('username')

# See first 10 rows
bad_pwd_data.head(10)

,id,username,password,invalid_password
931,932,abdul.rowland,0Vg3f0'L,True
713,714,addie.cherry,ancan,True
857,858,adele.moreno,76;%}C^3J,True
291,292,adeline.bush,"ejcKv=""ITF",True
663,664,adolfo.kane,6agy:loY!,True
775,776,adolfo.lara,YXJZF\/|W,True
51,52,ahmad.hopper,foster,True
298,299,aida.combs,"&1,y23?",True
898,899,aisha.jenkins,)_)#3(`5.~3-,True
471,472,al.dunlap,sanaang,True


In [23]:
# Question 1: What percentage of users have invalid passwords?
bad_pass = round(len(bad_pwd_data)/len(logins),2)
bad_pass

0.75

In [24]:
# Question 2: Which users need to change their passwords?
email_list = bad_pwd_data.username
email_list

931           abdul.rowland
713            addie.cherry
857            adele.moreno
291            adeline.bush
663             adolfo.kane
775             adolfo.lara
51             ahmad.hopper
298              aida.combs
898           aisha.jenkins
471               al.dunlap
356            alana.franco
546         alberta.leblanc
306            alec.robbins
831    alejandra.stephenson
44          alejandro.burke
195        alejandro.nieves
483        alexander.thomas
920       alexandria.hinton
93        alexis.mccullough
219         alexis.reynolds
456          alfonso.weaver
366           alfonzo.johns
595          alisa.campbell
781             alisa.cohen
442             alison.neal
452          allan.marshall
338           alonzo.fowler
751           amado.bridges
207        amado.fitzgerald
543           amber.summers
               ...         
64              ursula.wood
664       valentin.castillo
551           valeria.curry
0            vance.jennings
731           vaness